In [8]:
import pandas as pd
lazy_df=pd.read_csv("Employee.csv")
lazy_df.head()
#讀取資料集，顯示前五筆資料測試讀取是否成功
cat_columns =["Education", "City", "Gender", "EverBenched"]

from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
lazy_df_cat = encoder.fit_transform(lazy_df[cat_columns])
lazy_df[cat_columns] = lazy_df_cat
#把資料集中的類別變數整合成整數陣列，方便之後能快速存取
X = lazy_df.drop("LeaveOrNot", axis=1)
y = lazy_df["LeaveOrNot"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=8964,
    stratify=y
)
#決定我們要預測的對象(LeaveOrNot)，並把資料集八二分，留二十趴給測試，其他用來訓練
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=2000,      
    class_weight='balanced',  
    max_depth=18,           
    random_state=700,
    criterion='gini',
    max_features='sqrt'
)

#開始接觸種樹相關的設定了，調整數量、分類點的品管標準等等
rf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
y_train_pred = rf.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train accuracy:{}".format(train_accuracy))
#開始訓練，測試看他的準確性長什麼樣子
y_test_pred =rf.predict(X_test)
y_test_pred[:10]
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy:{}".format(test_accuracy))
#進行預測，順道看看他猜的準不準，82趴好像...堪用？
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1200)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

rf.fit(X_train_balanced, y_train_balanced)  # Train on balanced data
#SMOTE
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_test_pred, labels=[1, 0])
cm = pd.DataFrame(cm, columns = ["Predicted_1", "Predicted_0"], index = ["Actual_1", "Actual_0"])
cm.head()
#叫出混淆矩陣，看看精確的預測情況，尤其是偽陽、偽陰這兩種雷包到底存在多少
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print ("Accuracy: {}".format(round(accuracy, 3)))
print ("Precision: {}".format(round(precision, 3)))
print ("Recall: {}".format(round(recall, 3)))
print ("F1 Score: {}".format(round(f1, 3)))
#懶惰，所以用了package算準確率、精確率、召回率以及最重要的 F1 score，結果如下

Train accuracy:0.9368491770238495
Test accuracy:0.8174496644295302


/Applications/Utilities/anaconda3/envs/env/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Accuracy: 0.817
Precision: 0.761
Recall: 0.684
F1 Score: 0.72


In [4]:
#廢話這麼多，總算要進到最緊張的部分了，上面練完後，我們要來實測了
hw_df = pd.read_csv("Employee_test.csv")
hw_df_cat = encoder.fit_transform(hw_df[cat_columns])
hw_df[cat_columns] = hw_df_cat
#讀資料，順便把類別變數的部分弄好
hw_df_pred = rf.predict(hw_df)
submission = pd.read_csv('sample_submission.csv')
submission['LeaveOrNot'] = hw_df_pred
submission.to_csv("fourth.csv", index=False)